In [1]:
import pandas as pd

df = pd.read_csv('caso_full.csv')
df = df[df['state'] == 'SC']
df = df[df['place_type'] == 'city']
df

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1398211,Florianópolis,4205407.0,2020-03-12,202011,508826.0,500973.0,False,False,2,0.39306,2020-03-12,0.0000,0,1,city,SC,2,0
1398213,Florianópolis,4205407.0,2020-03-13,202011,508826.0,500973.0,False,False,2,0.39306,2020-03-13,0.0000,0,2,city,SC,0,0
1398214,Joinville,4209102.0,2020-03-13,202011,597658.0,590466.0,False,False,1,0.16732,2020-03-13,0.0000,0,1,city,SC,1,0
1398216,Florianópolis,4205407.0,2020-03-14,202011,508826.0,500973.0,False,False,2,0.39306,2020-03-14,0.0000,0,3,city,SC,0,0
1398217,Joinville,4209102.0,2020-03-14,202011,597658.0,590466.0,False,False,1,0.16732,2020-03-14,0.0000,0,2,city,SC,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492663,Zortéa,4219853.0,2021-03-21,202112,3398.0,3363.0,False,True,612,18010.59447,2021-03-20,0.0131,8,328,city,SC,0,0
1492664,Água Doce,4200408.0,2021-03-21,202112,7152.0,7145.0,False,True,579,8095.63758,2021-03-20,0.0069,4,323,city,SC,0,0
1492665,Águas Frias,4200556.0,2021-03-21,202112,2354.0,2366.0,False,True,435,18479.18437,2021-03-20,0.0000,0,300,city,SC,0,0
1492666,Águas Mornas,4200606.0,2021-03-21,202112,6559.0,6469.0,False,True,631,9620.36896,2021-03-20,0.0111,7,350,city,SC,0,0


In [2]:
from urllib.request import urlopen
import json
import gzip


with urlopen('https://servicodados.ibge.gov.br/api/v2/malhas/42?resolucao=5&formato=application/vnd.geo+json') as response:
    response = gzip.GzipFile(fileobj=response)
    counties = json.load(response)

for place in counties['features']:
    place['id'] = place['properties']['codarea']

counties['features'][0] 

{'type': 'Feature',
 'properties': {'codarea': '4200101',
  'centroide': [-52.256564545323165, -26.562096247120955]},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-52.1587, -26.4662],
     [-52.0983, -26.4835],
     [-52.0937, -26.5133],
     [-52.0813, -26.5251],
     [-52.0651, -26.523],
     [-52.0291, -26.5468],
     [-52.0283, -26.5933],
     [-52.0417, -26.5988],
     [-52.0672, -26.6781],
     [-52.0861, -26.6867],
     [-52.0886, -26.6687],
     [-52.1334, -26.6851],
     [-52.1476, -26.6717],
     [-52.1449, -26.6928],
     [-52.1245, -26.7249],
     [-52.1716, -26.7242],
     [-52.1847, -26.7132],
     [-52.1801, -26.6803],
     [-52.1987, -26.6631],
     [-52.2337, -26.6803],
     [-52.2549, -26.6758],
     [-52.2808, -26.6437],
     [-52.3399, -26.6678],
     [-52.3565, -26.6598],
     [-52.3681, -26.6714],
     [-52.3932, -26.6838],
     [-52.4129, -26.6581],
     [-52.3927, -26.6375],
     [-52.4148, -26.6143],
     [-52.4183, -26.5863],
     [-52.4594, -26.

In [3]:
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
import plotly
import os
import plotly.graph_objects as go
import numpy as np
import math

date_format = "%Y-%m-%d"

cidades = df['city'].unique()

dia_cidade = {}
df_ordered = df.sort_values(by=['date'])
dates = df_ordered['date'].unique()

for d in dates:
    df_day = df_ordered[df_ordered['date'] == d]
    
    for i,r in df_day.iterrows():
        if r['city'] not in dia_cidade:
            dia_cidade[r['city']] = d
            
dia_cidade

values = []

for i,r in df_ordered.iterrows():
    hoje = datetime.strptime(r['date'], date_format)
    dia = datetime.strptime(dates[0], date_format)
    novo = hoje-dia
    
    mort_rate = (r['last_available_deaths']/r['estimated_population_2019'])*100000
    novos_rate = r['new_confirmed']/r['estimated_population_2019']*100000
    conf_rate = r['last_available_confirmed_per_100k_inhabitants']
    
    if(math.isnan(conf_rate)):
        conf_rate = 0
    
    if(math.isnan(novos_rate)):
        novos_rate = 0
    
    if(math.isnan(mort_rate)):
        mort_rate = 0

    values.append({'Dia': novo.days,
                    'Cod. IBGE': r['city_ibge_code'],
                    'Data': hoje,
                   'Confirmados': r['last_available_confirmed'],
                   'Confirmados para cada 100 mil habitantes': int(conf_rate),
                   'Letalidade': round(r['last_available_death_rate']*100, 2),
                   'Mortes': r['last_available_deaths'],
                   'Município': r['city'],
                   'População': r['estimated_population_2019'],
                   'Mortalidade': (r['last_available_deaths']/r['estimated_population_2019'])*100,
                   'Mortalidade para cada 100 mil habitantes': int(mort_rate),
                   'Novos Casos': r['new_confirmed'],
                   'Novos Casos para cada 100 mil habitantes': int(novos_rate),
                   'Taxa de Novos Casos': r['new_confirmed']/max(r['last_available_confirmed']-r['new_confirmed'],1)*100,
                   'Novos Obitos': r['new_deaths'],

                  })

df_processado = pd.DataFrame(values)
df_processado

,Dia,Cod. IBGE,Data,Confirmados,Confirmados para cada 100 mil habitantes,Letalidade,Mortes,Município,População,Mortalidade,Mortalidade para cada 100 mil habitantes,Novos Casos,Novos Casos para cada 100 mil habitantes,Taxa de Novos Casos,Novos Obitos
0,0,4205407.0,2020-03-12,2,0,0.00,0,Florianópolis,500973.0,0.000000,0,2,0,200.0,0
1,1,4205407.0,2020-03-13,2,0,0.00,0,Florianópolis,500973.0,0.000000,0,0,0,0.0,0
2,1,4209102.0,2020-03-13,1,0,0.00,0,Joinville,590466.0,0.000000,0,1,0,100.0,0
3,2,4205407.0,2020-03-14,2,0,0.00,0,Florianópolis,500973.0,0.000000,0,0,0,0.0,0
4,2,4209102.0,2020-03-14,1,0,0.00,0,Joinville,590466.0,0.000000,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94078,374,4206207.0,2021-03-21,1706,14736,1.17,20,Gravatal,11501.0,0.173898,173,0,0,0.0,0
94079,374,4206009.0,2021-03-21,2011,13768,0.75,15,Governador Celso Ramos,14471.0,0.103656,103,0,0,0.0,0
94080,374,4205902.0,2021-03-21,6030,8517,1.71,103,Gaspar,69639.0,0.147906,147,0,0,0.0,0
94081,374,4205704.0,2021-03-21,2647,11226,0.83,22,Garopaba,23078.0,0.095329,95,0,0,0.0,0


In [4]:
import cv2
import os

def makeVideo(dir, df, color, color_range, color_scale = "Viridis", code='Cod. IBGE'):
    dir_name = dir
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        os.mkdir(dir_name+'/src')
        os.mkdir(dir_name+'/src/video')
        
    dias = df['Dia'].unique().astype(np.int32)
    for d in dias:
        data = datetime.strptime(dates[0], date_format)
        data = data + timedelta(days=int(d))
        data = str(data.strftime(date_format))
        print(data)
        dft = df[df['Dia'] == d]
        fig = px.choropleth_mapbox(dft, geojson=counties, locations=code, color=dft[color],
                                color_continuous_scale=color_scale,
                                mapbox_style="carto-positron",
                                opacity=0.8,
                                range_color= color_range,
                                hover_name="Município", hover_data=["Confirmados", 'População'],
                                    zoom=6.2, center = {"lat": -27.4, "lon": -51}
                                )

        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title=data)
        fig.update_layout(
            autosize=False,
            width=800,
            height=600,)
        fig.write_image(dir_name+'/src/video/'+data+'.png')
    
    image_folder = dir_name+'/src/video'
    video_name = dir_name+'/'+dir+'.avi'
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 10, (width,height))

    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) 
    fontScale = .75
    color = (30, 30, 30) 
    thickness = 1

    for image in images:
        img = cv2.imread(os.path.join(image_folder, image))
        img = cv2.putText(img, image[:-4], org, font, fontScale, color, thickness, cv2.LINE_AA) 
        video.write(img)

    cv2.destroyAllWindows()
    video.release()

In [10]:
# makeVideo('casos', df_processado, 'Confirmados para cada 100 mil habitantes', [0,17000], color_scale = "Plasma")
# makeVideo('letalidade', df_processado, 'Letalidade', [0,2.42], color_scale = "Plasma")
# makeVideo('mortalidade', df_processado, 'Mortalidade para cada 100 mil habitantes', [0,130], color_scale = "Plasma")

In [5]:
def makeMapChart(dir, df, color, color_range, color_scale = "Viridis", code='Cod. IBGE'):
    dir_name = dir
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
        os.mkdir(dir_name+'/src')

    dias = df['Dia'].unique().astype(np.int32)
    d = dias[-1]
    data = datetime.strptime(dates[0], date_format)
    data = data + timedelta(days=int(d))
    data = str(data.strftime(date_format))
    print(data)
    dft = df[df['Dia'] == d]
    fig = px.choropleth_mapbox(dft, geojson=counties, locations=code, color=dft[color],
                            color_continuous_scale=color_scale,
                            mapbox_style="carto-positron",
                            opacity=0.8,
                            range_color= color_range,
                            hover_name="Município", hover_data=["Novos Casos","Confirmados","Confirmados para cada 100 mil habitantes", "Letalidade", "Novos Obitos", "Mortes", "Mortalidade para cada 100 mil habitantes", 'População'],
                                zoom=5.8, center = {"lat": -27.4, "lon": -51}
                            )

    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title=data)
    plotly.offline.plot(fig, filename=(dir_name+'/src/last.html'))

In [12]:
makeMapChart('casos', df_processado, 'Confirmados para cada 100 mil habitantes', [0,17000], color_scale = "Plasma")
makeMapChart('letalidade', df_processado, 'Letalidade', [0,2.42], color_scale = "Plasma")
makeMapChart('mortalidade', df_processado, 'Mortalidade para cada 100 mil habitantes', [0,130], color_scale = "Plasma")

2021-03-21
2021-03-21
2021-03-21


In [6]:
import numpy as np

outros_municipios_interesse = [
    'Florianópolis','Joinville','Balneário Camboriú','São José','Navegantes','Criciúma','Chapecó','Itajaí','Blumenau','Biguaçu'
]

municipios_amfri = [
    'Balneário Camboriú','Balneário Piçarras','Bombinhas','Camboriú','Itajaí',
    'Ilhota','Itapema', 'Luiz Alves','Navegantes','Penha','Porto Belo'
]

temp = np.concatenate((outros_municipios_interesse, municipios_amfri))
temp = np.unique(temp)
municipios_interesse = temp.tolist()

# municipios_interesse
df_interesse = df_processado[df_processado['Município'].isin(municipios_interesse)]

main_cols = ["Novos Casos","Confirmados","Confirmados para cada 100 mil habitantes", "Letalidade", "Novos Obitos", "Mortes", "Mortalidade para cada 100 mil habitantes", 'População']

In [9]:
def makeLineChart(name, row, is_log = False):
    fig = px.line(df_interesse, x="Data", y=row, color='Município', hover_name="Município", hover_data=main_cols, log_y=is_log)
    plotly.offline.plot(fig, filename=(name+'.html'))

In [10]:
makeLineChart('avanco_casos', "Confirmados para cada 100 mil habitantes")
makeLineChart('letalidade', "Letalidade", is_log=True)
makeLineChart('mortalidade', "Mortalidade para cada 100 mil habitantes")


In [16]:
import plotly.graph_objects as go

media_dias = 14
df_municipio = df_interesse[df_interesse["Município"] == "Itajaí"]

novos = []
obitos = []
out = []
last = 0
last_obitos = 0
for i,r in df_municipio.iterrows():
    
    if(r['Novos Casos'] > 0):
        novos.append(r['Novos Casos'])
        if(len(novos) > media_dias):
            novos.pop(0)

        obitos.append(r['Novos Obitos'])
        if(len(obitos) > media_dias):
            obitos.pop(0)

        out.append({
            "Data": r['Data'],
            "Mes": r['Data'].strftime("%Y-%m"),
            "Casos Novos": r['Novos Casos'],
            "Novos Obitos": r['Novos Obitos'],
            'Novos Casos': np.mean(np.array(novos))-last,
            "Obitos": np.mean(np.array(obitos))-last_obitos
        })
        last = np.mean(np.array(novos))
        last_obitos = np.mean(np.array(obitos))
mun = pd.DataFrame(out)
mun



,Data,Mes,Casos Novos,Novos Obitos,Novos Casos,Obitos
0,2020-03-21,2020-03,6,0,6.000000,0.000000
1,2020-03-22,2020-03,1,0,-2.500000,0.000000
2,2020-03-24,2020-03,3,0,-0.166667,0.000000
3,2020-03-25,2020-03,2,0,-0.333333,0.000000
4,2020-03-26,2020-03,4,0,0.200000,0.000000
...,...,...,...,...,...,...
336,2021-03-16,2021-03,98,1,2.928571,-0.357143
337,2021-03-17,2021-03,149,7,3.500000,0.357143
338,2021-03-18,2021-03,170,7,6.000000,0.428571
339,2021-03-19,2021-03,202,6,11.071429,0.285714


In [17]:


df_amfri = df_processado[df_processado["Município"].isin(municipios_amfri)]

novos = []
obitos = []
out = []
last = 0
last_obitos = 0
for dia in df_amfri['Data'].unique():
    df_dia = df_amfri[df_amfri['Data'] == dia]

    c = 0
    o = 0
    for i,r in df_dia.iterrows():
        c+=r['Novos Casos']
        o+=r['Novos Obitos']
    if(c>0):
        novos.append(c)
        if(len(novos) > media_dias):
            novos.pop(0)

        obitos.append(o)
        if(len(obitos) > media_dias):
            obitos.pop(0)

        out.append({
            "Data": r['Data'],
            "Mes": r['Data'].strftime("%Y-%m"),
            "Casos Novos": c,
            "Novos Obitos": o,
            'Novos Casos': np.mean(np.array(novos))-last,
            "Obitos": np.mean(np.array(obitos))-last_obitos
        })
        last = np.mean(np.array(novos))
        last_obitos = np.mean(np.array(obitos))

amfri = pd.DataFrame(out)
amfri

,Data,Mes,Casos Novos,Novos Obitos,Novos Casos,Obitos
0,2020-03-18,2020-03,2,0,2.000000,0.000000
1,2020-03-19,2020-03,2,0,0.000000,0.000000
2,2020-03-20,2020-03,1,0,-0.333333,0.000000
3,2020-03-21,2020-03,8,0,1.583333,0.000000
4,2020-03-22,2020-03,1,0,-0.450000,0.000000
...,...,...,...,...,...,...
355,2021-03-16,2021-03,474,11,10.642857,-0.142857
356,2021-03-17,2021-03,441,19,-0.928571,0.928571
357,2021-03-18,2021-03,483,21,0.285714,1.071429
358,2021-03-19,2021-03,538,20,16.714286,1.000000


In [18]:
df_sc = df_processado.copy()

novos = []
obitos = []
out = []
last = 0
last_obitos = 0
for dia in df_sc['Data'].unique():
    df_dia = df_sc[df_sc['Data'] == dia]

    c = 0
    o = 0
    for i,r in df_dia.iterrows():
        c+=r['Novos Casos']
        o+=r['Novos Obitos']
    if(c>0):
        novos.append(c)
        if(len(novos) > media_dias):
            novos.pop(0)

        obitos.append(o)
        if(len(obitos) > media_dias):
            obitos.pop(0)

        out.append({
            "Data": r['Data'],
            "Mes": r['Data'].strftime("%Y-%m"),
            "Casos Novos": c,
            "Novos Obitos": o,
            'Novos Casos': np.mean(np.array(novos))-last,
            "Obitos": np.mean(np.array(obitos))-last_obitos
        })
        last = np.mean(np.array(novos))
        last_obitos = np.mean(np.array(obitos))

sc = pd.DataFrame(out)
sc

,Data,Mes,Casos Novos,Novos Obitos,Novos Casos,Obitos
0,2020-03-12,2020-03,2,0,2.000000,0.000000
1,2020-03-13,2020-03,1,0,-0.500000,0.000000
2,2020-03-14,2020-03,2,0,0.166667,0.000000
3,2020-03-15,2020-03,1,0,-0.166667,0.000000
4,2020-03-16,2020-03,1,0,-0.100000,0.000000
...,...,...,...,...,...,...
364,2021-03-16,2021-03,7547,167,183.785714,6.214286
365,2021-03-17,2021-03,5764,163,-3.571429,5.500000
366,2021-03-18,2021-03,5440,126,-126.357143,2.285714
367,2021-03-19,2021-03,4947,134,-51.928571,3.071429


In [19]:
def makeMobileMean(frame, col, name):
    fig = go.Figure(go.Waterfall(
        name=col,
        orientation = "v",
        decreasing = {"marker":{"color":"green",}},
        increasing = {"marker":{"color":"red"}},
        x = frame["Data"],
        y = frame[col],
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ))
    
    plotly.offline.plot(fig, filename=(name+'.html'))

In [20]:
makeMobileMean(mun, 'Novos Casos', 'medias/casos_itj')
makeMobileMean(mun, 'Obitos', 'medias/obitos_itj')

makeMobileMean(amfri, 'Novos Casos', 'medias/casos_amfri')
makeMobileMean(amfri, 'Obitos', 'medias/obitos_amfri')

makeMobileMean(sc, 'Novos Casos', 'medias/casos_sc')
makeMobileMean(sc, 'Obitos', 'medias/obitos_sc')

In [21]:
import plotly.express as px

def makeBox(frame, col, name):
    sc_20 = frame[frame['Mes'].isin(['2021-01','2021-02','2021-03'])]
    sc_20['Data'] = sc_20['Data'].dt.strftime('%Y-%m-%d')
    fig = px.box(sc_20, color="Mes", y=col, points="all", hover_data=["Data"])
    plotly.offline.plot(fig, filename=(name+'.html'))

In [22]:
makeBox(sc, 'Casos Novos', 'box/novos_casos_sc')
makeBox(amfri, 'Casos Novos', 'box/novos_casos_amfri')
makeBox(mun, 'Casos Novos', 'box/novos_casos_itj')

makeBox(sc, 'Novos Obitos', 'box/novos_obitos_sc')
makeBox(amfri, 'Novos Obitos', 'box/novos_obitos_amfri')
makeBox(mun, 'Novos Obitos', 'box/novos_obitos_itj')

<ipython-input-21-c782b05e926f>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

